In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Category:Neighbourhoods in Delhi - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YCOQZApAIC8AADuip1wAAACM","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Neighbourhoods_in_Delhi","wgTitle":"Neighbourhoods in Delhi","wgCurRevisionId":975675815,"wgRevisionId":975675815,"wgArticleId":3822058,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category link from Wikidata","Geography of Delhi","Neighbourhoods in India by state or union terri

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
dl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

dl_df.head()

,Neighborhood
0,List of neighbourhoods of Delhi
1,Ashok Nagar (Delhi)
2,Ashok Vihar
3,Ashram Chowk
4,Babarpur


In [8]:
dl_df.shape

(153, 1)

## Getting the geographical cordinates.

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{},Delhi,India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in dl_df["Neighborhood"].tolist() ]

In [10]:
coords

[[28.523450000000025, 77.26178000000004],
 [28.692230000000052, 77.30124000000006],
 [28.69037000000003, 77.17609000000004],
 [28.710598435255907, 77.32696519316737],
 [28.50738000000007, 77.30346000000003],
 [28.50738000000007, 77.30346000000003],
 [28.65223022436032, 77.12941079026544],
 [28.800590000000057, 77.03473000000008],
 [28.549540000000036, 77.18167000000005],
 [28.699880000000064, 77.25906000000003],
 [28.595060000000046, 77.18573000000004],
 [28.656270000000063, 77.23232000000007],
 [28.67671000000007, 77.21767000000006],
 [28.633940000000052, 77.21968000000004],
 [28.60761000000008, 77.08714000000003],
 [28.654597885415757, 77.2333966005242],
 [28.62832000000003, 77.24727000000007],
 [28.60486000000003, 77.08511000000004],
 [28.71768000000003, 77.26517000000007],
 [28.560590000000047, 77.24678000000006],
 [28.57298000000003, 77.23357000000004],
 [28.591510000000028, 77.12945000000008],
 [28.699110000000076, 77.19105000000008],
 [28.592377945217475, 77.1594798751621],
 [28

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# merge the coordinates into the original dataframe
dl_df['Latitude'] = df_coords['Latitude']
dl_df['Longitude'] = df_coords['Longitude']

In [17]:
# check the neighborhoods and the coordinates
print(dl_df.shape)
dl_df


(153, 3)


,Neighborhood,Latitude,Longitude
0,List of neighbourhoods of Delhi,28.523450,77.261780
1,Ashok Nagar (Delhi),28.692230,77.301240
2,Ashok Vihar,28.690370,77.176090
3,Ashram Chowk,28.710598,77.326965
4,Babarpur,28.507380,77.303460
5,"Badarpur, Delhi",28.507380,77.303460
6,Bali Nagar,28.652230,77.129411
7,Bawana,28.800590,77.034730
8,Ber Sarai,28.549540,77.181670
9,Bhajanpura,28.699880,77.259060


In [13]:
# save the DataFrame as CSV file
dl_df.to_csv("kl_df.csv", index=False)

## Creating a map of Delhi with neighbourhood superimposed on it.

In [14]:
# get the coordinates of Delhi
address = 'Delhi , India'

geolocator = Nominatim(user_agent="gauravvajpayee99@gmail.com")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, india {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, india 28.6517178, 77.2219388.


In [15]:
# create map of Delhi using latitude and longitude values
map_dl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dl)  
    
map_dl


In [16]:
# save the map as HTML file
map_dl.save('map_dl.html')

## Getting Venue Data from Foursquare

In [17]:
# define Foursquare Credentials and Version
CLIENT_ID = '2JE15UX221MJBOHVPUVJIUHNCRIZUAIBRHPTHMH41AS31ND2' # your Foursquare ID
CLIENT_SECRET = 'B4TVHPDUMYNGGV4M11PVNGITEQZIOXMOWCJ2OP04ZW1UHREL' # your Foursquare Secret
VERSION = '20210210' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2JE15UX221MJBOHVPUVJIUHNCRIZUAIBRHPTHMH41AS31ND2
CLIENT_SECRET:B4TVHPDUMYNGGV4M11PVNGITEQZIOXMOWCJ2OP04ZW1UHREL


In [18]:
radius = 2000 # Top 100 venues in radius of 2000m
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [21]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(20)

(6335, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of neighbourhoods of Delhi,28.52345,77.26178,Khana Khazana,28.526114,77.259298,Restaurant
1,List of neighbourhoods of Delhi,28.52345,77.26178,City Of Joy,28.532685,77.253003,Restaurant
2,List of neighbourhoods of Delhi,28.52345,77.26178,Culinaire,28.530777,77.245816,Thai Restaurant
3,List of neighbourhoods of Delhi,28.52345,77.26178,CR Park Market No. 2,28.536463,77.253386,Market
4,List of neighbourhoods of Delhi,28.52345,77.26178,Olympia Gymnasium & Spa,28.527990,77.245713,Gym
5,List of neighbourhoods of Delhi,28.52345,77.26178,Crowne Plaza,28.528669,77.272257,Hotel
6,List of neighbourhoods of Delhi,28.52345,77.26178,Domino's Pizza,28.530000,77.272000,Pizza Place
7,List of neighbourhoods of Delhi,28.52345,77.26178,Qureshi's,28.527830,77.246837,BBQ Joint
8,List of neighbourhoods of Delhi,28.52345,77.26178,Subway,28.540065,77.259129,Sandwich Place
9,List of neighbourhoods of Delhi,28.52345,77.26178,Barista,28.534018,77.248300,Coffee Shop


In [22]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashok Nagar (Delhi),4,4,4,4,4,4
Ashok Vihar,21,21,21,21,21,21
Ashram Chowk,4,4,4,4,4,4
Babarpur,7,7,7,7,7,7
"Badarpur, Delhi",7,7,7,7,7,7
Bali Nagar,48,48,48,48,48,48
Bawana,1,1,1,1,1,1
Ber Sarai,94,94,94,94,94,94
Bhajanpura,6,6,6,6,6,6


In [23]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Restaurant', 'Thai Restaurant', 'Market', 'Gym', 'Hotel',
       'Pizza Place', 'BBQ Joint', 'Sandwich Place', 'Coffee Shop',
       'Fast Food Restaurant', 'Castle', 'Smoke Shop', 'Park',
       'Train Station', 'Athletics & Sports', 'Asian Restaurant',
       'Snack Place', 'Indian Restaurant', 'Department Store',
       'South Indian Restaurant', 'Dessert Shop', 'Basketball Court',
       'Light Rail Station', 'ATM', 'Print Shop', 'Indian Sweet Shop',
       'Café', 'American Restaurant', 'Bakery', 'Donut Shop', 'Diner',
       'Hookah Bar', 'Pub', 'Sports Bar', 'Garden Center', 'Multiplex',
       'Shopping Mall', 'Furniture / Home Store', 'Fried Chicken Joint',
       'Bar', 'Lounge', 'Cosmetics Shop', 'Garden', 'Convenience Store',
       'Food Truck', 'Playground', 'Art Gallery', 'Tibetan Restaurant',
       'Tea Room', 'Ice Cream Shop'], dtype=object)

In [24]:
# one hot encoding
dl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dl_onehot.columns[-1]] + list(dl_onehot.columns[:-1])
dl_onehot = dl_onehot[fixed_columns]

print(dl_onehot.shape)
dl_onehot.head()

(6335, 214)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Campground,Castle,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio,Zoo
0,List of neighbourhoods of Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,List of neighbourhoods of Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,List of neighbourhoods of Delhi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [26]:
dl_grouped = dl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(dl_grouped.shape)
dl_grouped

(152, 214)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Campground,Castle,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio,Zoo
0,Ashok Nagar (Delhi),0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [27]:
len(dl_grouped[dl_grouped["Park"] > 0])

58

In [28]:
dl_park = dl_grouped[["Neighborhoods","Park"]]

In [29]:
dl_park.head()

,Neighborhoods,Park
0,Ashok Nagar (Delhi),0.5
1,Ashok Vihar,0.0
2,Ashram Chowk,0.0
3,Babarpur,0.0
4,"Badarpur, Delhi",0.0


7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Delhi into 3 clusters.

In [30]:
#set number of clusters
kclusters = 3

dl_clustering = dl_park.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [31]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
dl_merged = dl_park.copy()

# add clustering labels
dl_merged["Cluster Labels"] = kmeans.labels_

In [33]:

dl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
dl_merged.head()

,Neighborhood,Park,Cluster Labels
0,Ashok Nagar (Delhi),0.5,1
1,Ashok Vihar,0.0,0
2,Ashram Chowk,0.0,0
3,Babarpur,0.0,0
4,"Badarpur, Delhi",0.0,0


In [34]:
# merge delhi_grouped with delhi_data to add latitude/longitude for each neighborhood
dl_merged = dl_merged.join(dl_df.set_index("Neighborhood"), on="Neighborhood")

print(dl_merged.shape)
dl_merged.head() # check the last columns!

(152, 5)


,Neighborhood,Park,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.5,1,28.692230,77.301240
1,Ashok Vihar,0.0,0,28.690370,77.176090
2,Ashram Chowk,0.0,0,28.710598,77.326965
3,Babarpur,0.0,0,28.507380,77.303460
4,"Badarpur, Delhi",0.0,0,28.507380,77.303460


In [42]:
# sort the results by Cluster Labels
print(dl_merged.shape)
dl_merged.sort_values(["Cluster Labels"], inplace=True)
dl_merged

(152, 5)


,Neighborhood,Park,Cluster Labels,Latitude,Longitude
75,Meera Bagh,0.000000,0,28.661210,77.086900
96,Okhla,0.000000,0,28.532470,77.278390
95,Nizamuddin West,0.022222,0,28.588650,77.244860
139,Sriniwaspuri,0.000000,0,28.565680,77.257330
93,Nigambodh Ghat,0.000000,0,28.664710,77.236330
92,New Moti Bagh,0.022989,0,28.581010,77.181840
91,New Friends Colony,0.000000,0,28.578120,77.269990
90,New Delhi,0.000000,0,28.630950,77.217210
89,"Netaji Nagar, Delhi",0.017857,0,28.577460,77.185170
140,Tilak Nagar (Delhi),0.000000,0,28.640890,77.089220


In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dl_merged['Latitude'], dl_merged['Longitude'], dl_merged['Neighborhood'], dl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

In [36]:
dl_merged.loc[dl_merged['Cluster Labels'] == 0] # cluster 0

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
1,Ashok Vihar,0.000000,0,28.690370,77.176090
2,Ashram Chowk,0.000000,0,28.710598,77.326965
3,Babarpur,0.000000,0,28.507380,77.303460
4,"Badarpur, Delhi",0.000000,0,28.507380,77.303460
5,Bali Nagar,0.020833,0,28.652230,77.129411
6,Bawana,0.000000,0,28.800590,77.034730
7,Ber Sarai,0.010638,0,28.549540,77.181670
8,Bhajanpura,0.000000,0,28.699880,77.259060
9,Chanakyapuri,0.025641,0,28.595060,77.185730
10,Chandni Chowk,0.000000,0,28.656270,77.232320


In [37]:
dl_merged.loc[dl_merged['Cluster Labels'] == 1] # cluster 1

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.500000,1,28.692230,77.301240
42,"Jaffrabad, Delhi",0.285714,1,28.678705,77.272822
88,Naveen Shahdara,0.400000,1,28.673690,77.283260
124,Seelampur subdivision,0.333333,1,28.689440,77.293810
125,Seemapuri subdivision,0.250000,1,28.687600,77.330620


In [38]:
dl_merged.loc[dl_merged['Cluster Labels'] == 2] # cluster 2

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
22,Dhaula Kuan,0.054054,2,28.592378,77.159480
23,Dilshad Colony,0.166667,2,28.684700,77.327740
28,"Gadaipur, Mehrauli, New Delhi",0.062500,2,28.521130,77.180170
38,Haqiqat Nagar,0.125000,2,30.919800,75.809450
52,Kathputli Colony,0.050000,2,28.653460,77.152860
57,Kingsway Camp,0.095238,2,28.711690,77.201970
67,Madanpur Khadar JJ Colony,0.125000,2,28.533280,77.316450
76,Mehrauli,0.062500,2,28.521130,77.180170
78,Moti Bagh,0.076923,2,28.583620,77.164740
86,Naraina Vihar,0.071429,2,28.629760,77.133120


## By clustering the neighborhoods into 3 categories based on the frequency of occurrence of “ Parks” we got following resulting clusters.
## Cluster 0 - neighborhoods falling in this category are have very less frequency of occurrence of Parks.
## Clusters 1 - neighborhood falling in this category have high frequency of occurrence of Parks.
## Clusters2-neighborhood falling in this category have moderate(between high and low) frequency of occurrence of Parks. 